<a href="https://colab.research.google.com/github/TeamBJEJ/Natural-Language-Processing-with-Disaster-tweet/blob/main/test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install kaggle
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"bumjoonpark","key":"0f521badc01ac767524ddcde4c8b7f7a"}'}

In [ ]:
!mkdir -p ~/.kaggle

In [ ]:
!cp kaggle.json ~/.kaggle/ 

In [ ]:
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
ls -1ha kaggle.json

kaggle.json


In [ ]:
!kaggle competitions list

ref                                            deadline             category            reward  teamCount  userHasEntered  
---------------------------------------------  -------------------  ---------------  ---------  ---------  --------------  
contradictory-my-dear-watson                   2030-07-01 23:59:00  Getting Started     Prizes         94           False  
gan-getting-started                            2030-07-01 23:59:00  Getting Started     Prizes        199           False  
tpu-getting-started                            2030-06-03 23:59:00  Getting Started  Knowledge        457           False  
digit-recognizer                               2030-01-01 00:00:00  Getting Started  Knowledge       3090           False  
titanic                                        2030-01-01 00:00:00  Getting Started  Knowledge      24864           False  
house-prices-advanced-regression-techniques    2030-01-01 00:00:00  Getting Started  Knowledge       6178           False  
connectx

In [ ]:
!kaggle competitions download -c nlp-getting-started

  0% 0.00/965k [00:00<?, ?B/s]
100% 965k/965k [00:00<00:00, 62.9MB/s]
  0% 0.00/411k [00:00<?, ?B/s]
100% 411k/411k [00:00<00:00, 127MB/s]
  0% 0.00/22.2k [00:00<?, ?B/s]
100% 22.2k/22.2k [00:00<00:00, 22.6MB/s]


## Load Datasets

In [ ]:
!ls

sample_data


In [ ]:
!ls

sample_data


In [ ]:
import pandas as pd 

In [ ]:
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

In [ ]:
len(train_df) * 0.8

6090.400000000001

In [ ]:
train_data = train_df[:6090]
valid_data = train_df[6090:]
print(len(train_data))
print(len(valid_data))

6090
1523


In [ ]:
train_data.to_csv('train_data.csv')
valid_data.to_csv('valid_data.csv')
test_df.to_csv('test_data.csv')

In [ ]:
test_data = test_df

# Prepare data with Torch

2-1 Fotmatting torch

In [ ]:
!pip3 install transformers

     |████████████████████████████████| 2.0MB 17.3MB/s 
     |████████████████████████████████| 890kB 53.3MB/s 
     |████████████████████████████████| 3.2MB 47.5MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp37-none-any.whl size=893262 sha256=11d64e9b5dd868943f31080ee924b600b65ce35c6fe92bc76690aa77ce51567c
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [ ]:
from transformers import AutoModel, BertTokenizer
tokenizer = BertTokenizer.from_pretrained('vinai/bertweet-base', use_fast = False)

Special tokens have been added in the vocabulary, make sure the associated word embedding are fine-tuned or trained.


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

import torchtext
from torchtext.legacy.data.dataset import TabularDataset, Dataset
from torchtext.legacy.data import Field, LabelField, BucketIterator
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

import numpy as np

import random
import math
import time

In [ ]:
train_data

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1
...,...,...,...,...,...
6085,8692,sinking,"Cypress, CA 90630",Do you feel like you are sinking in low self-i...,0
6086,8693,sinking,Sacramento,After a Few Years Afloat Pension Plans Start S...,1
6087,8694,sinking,NaN,Do you feel like you are sinking in unhappines...,0
6088,8695,sinking,"Vancouver, British Columbia",With a sinking music video tv career Brooke Ho...,0


In [ ]:
test_data

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan
...,...,...,...,...
3258,10861,NaN,NaN,EARTHQUAKE SAFETY LOS ANGELES ÛÒ SAFETY FASTE...
3259,10865,NaN,NaN,Storm in RI worse than last hurricane. My city...
3260,10868,NaN,NaN,Green Line derailment in Chicago http://t.co/U...
3261,10874,NaN,NaN,MEG issues Hazardous Weather Outlook (HWO) htt...


In [ ]:
X = np.concatenate([train_data.text.values, valid_data.text.values])
y = np.concatenate([train_data.target.values, valid_data.target.values])

X_train = train_data.text.values
X_val = valid_data.text.values
y_train = train_data.target.values
y_val = valid_data.target.values

In [ ]:
X_train

array(['Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all',
       'Forest fire near La Ronge Sask. Canada',
       "All residents asked to 'shelter in place' are being notified by officers. No other evacuation or shelter in place orders are expected",
       ...,
       'Do you feel like you are sinking in unhappiness? Take the quiz: http://t.co/OrJb3j803F http://t.co/MWdHXYfrag',
       'With a sinking music video tv career Brooke Hogan should be THANKING her Dad for the free publicity...although I doubt it will help her.',
       "@supernovalester I feel so bad for them. I can literally feel that feeling of your heart sinking bc you didn't get anyone ugh jfc"],
      dtype=object)

In [ ]:
y_val

array([0, 1, 0, ..., 1, 1, 1])

In [ ]:
test_data.sample(5)

,id,keyword,location,text
2209,7387,obliterate,NaN,(rubs lamp greets genie) just need the 1 wish ...
2324,7778,police,Philadelphia & Worldwide,Police: Tennessee theater shooting suspect was...
1911,6442,injured,The North Coast,@NewsBreaker @sothathappened SO wheres the sto...
363,1169,blight,The other side of the mirror,This is the Red Eye crossing the ocean\nPeelin...
2614,8730,sinking,"Washington, DC",Sinking. {part 2}\n??????\n#buildingmuseum #Th...


In [ ]:
if torch.cuda.is_available():
  device = torch.device('cuda')
  print(f'There are {torch.cuda.device_count()} GPU(s) available.')
  print('Device name:', torch.cuda.get_device_name(0))
else:
  print('No GPU availalbe, using the CPU instead')
  device = torch.device('cpu')

There are 1 GPU(s) available.
Device name: Tesla T4


In [ ]:
def preprocessing_for_bert(data):
    """Perform required preprocessing steps for pretrained BERT.
    @param    data (np.array): Array of texts to be processed.
    @return   input_ids (torch.Tensor): Tensor of token ids to be fed to a model.
    @return   attention_masks (torch.Tensor): Tensor of indices specifying which
                  tokens should be attended to by the model.
    """
    # Create empty lists to store outputs
    input_ids = []
    attention_masks = []

    # For every sentence...
    for sent in data:
        # `encode_plus` will:
        #    (1) Tokenize the sentence
        #    (2) Add the `[CLS]` and `[SEP]` token to the start and end
        #    (3) Truncate/Pad sentence to max length
        #    (4) Map tokens to their IDs
        #    (5) Create attention mask
        #    (6) Return a dictionary of outputs
        encoded_sent = tokenizer.encode_plus(
            text=sent,  # Preprocess sentence
            add_special_tokens=True,        # Add `[CLS]` and `[SEP]`
            max_length=MAX_LEN,                  # Max length to truncate/pad
            pad_to_max_length=True,         # Pad sentence to max length
            #return_tensors='pt',           # Return PyTorch tensor
            return_attention_mask=True      # Return attention mask
            )
        
        # Add the outputs to the lists
        input_ids.append(encoded_sent.get('input_ids'))
        attention_masks.append(encoded_sent.get('attention_mask'))

    # Convert lists to tensors
    print(input_ids)
    input_ids = torch.tensor(input_ids)
    attention_masks = torch.tensor(attention_masks)

    return input_ids, attention_masks

In [ ]:
all_comments = np.concatenate([train_data.text.values, valid_data.text.values, test_data.text.values])

In [ ]:
encoded_comments = [tokenizer.encode(sent, add_special_tokens = True) for sent in all_comments]

max_len = max([len(sent) for sent in encoded_comments])
print('Max length: ', max_len)

Max length:  78


In [ ]:
X

array(['Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all',
       'Forest fire near La Ronge Sask. Canada',
       "All residents asked to 'shelter in place' are being notified by officers. No other evacuation or shelter in place orders are expected",
       ...,
       'M1.94 [01:04 UTC]?5km S of Volcano Hawaii. http://t.co/zDtoyd8EbJ',
       'Police investigating after an e-bike collided with a car in Little Portugal. E-bike rider suffered serious non-life threatening injuries.',
       'The Latest: More Homes Razed by Northern California Wildfire - ABC News http://t.co/YmY4rSkQ3d'],
      dtype=object)

In [ ]:
MAX_LEN = 100

token_ids_0 = list(preprocessing_for_bert([X[0]])[0].squeeze().numpy())
token_text_0 = tokenizer.tokenize(X[0])
print('Original: ', X[0])
print('Tokenized: ', token_text_0)
print('Token IDs: ', token_ids_0)

token_ids_10 = list(preprocessing_for_bert([X[10]])[0].squeeze().numpy())
toekn_text_10 = tokenizer.tokenize(X[10])
print('Original: ', X[10])
print('Tokenized: ', toekn_text_10)
print('Token IDs: ', token_ids_10)

print('Tokenizing data...')
train_inputs, train_masks = preprocessing_for_bert(X_train)
val_inputs, val_masks = preprocessing_for_bert(X_val)
print('Complete!')


[[63998, None, None, None, None, None, None, None, None, None, None, None, None, None, None, 63996, 63997, 63997, 63997, 63997, 63997, 63997, 63997, 63997, 63997, 63997, 63997, 63997, 63997, 63997, 63997, 63997, 63997, 63997, 63997, 63997, 63997, 63997, 63997, 63997, 63997, 63997, 63997, 63997, 63997, 63997, 63997, 63997, 63997, 63997, 63997, 63997, 63997, 63997, 63997, 63997, 63997, 63997, 63997, 63997, 63997, 63997, 63997, 63997, 63997, 63997, 63997, 63997, 63997, 63997, 63997, 63997, 63997, 63997, 63997, 63997, 63997, 63997, 63997, 63997, 63997, 63997, 63997, 63997, 63997, 63997, 63997, 63997, 63997, 63997, 63997, 63997, 63997, 63997, 63997, 63997, 63997, 63997, 63997, 63997]]


/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2074: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


RuntimeError: ignored